In [ ]:
# Importing Libraries

In [ ]:
from distributed import Client
client = Client()

In [ ]:
import torch
import intel_extension_for_pytorch as ipex
import time
import evaluate
import modin.pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import random

In [ ]:
# loading training dataset

In [ ]:
excel_path = 'train_dataset.xlsx'
df = pd.read_excel(excel_path)

In [ ]:
df.head()

In [ ]:
# processing data

In [ ]:
df['summary'] =  df['summary'].astype(str)

In [ ]:
df['text'] =  df['text'].astype(str)

In [ ]:
data_dict = df.to_dict('list')  # convert DataFrame to dictionary of lists
dataset = Dataset.from_dict(data_dict)  # create Dataset object

In [ ]:
ds = dataset.train_test_split(test_size=2)

In [ ]:
ds["train"][0]

In [ ]:
def format_instruction(text: str, summary: str):
	return f"""### Instruction:
Summarize the following case details
### Input:
{text.strip()}

### Summary:
{summary.strip()}
""".strip()

In [ ]:
def generate_instruction_dataset(data_point):

    return {
        "text": data_point["text"],
        "summary": data_point["summary"],
        "text": format_instruction(data_point["text"],data_point["summary"])
    }

In [ ]:
def process_dataset(data: Dataset):
    return (
        data.shuffle(seed=42)
        .map(generate_instruction_dataset)
    )

In [ ]:
ds["train"] = process_dataset(ds["train"])
ds["test"] = process_dataset(ds["test"])

In [ ]:
ds["train"]["text"][0]

In [ ]:
# loading the LLM models

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id =  "meta-llama/Llama-2-7b-hf"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"


In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():

        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
print(model)

In [ ]:
# Trainable parameters

In [ ]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"], #specific to Llama models.
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print_trainable_parameters(model)

In [ ]:
Output_Dir = "Output_Dir"

In [ ]:
# Fine tuning

In [ ]:
from transformers import TrainingArguments

training_arguments = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=1,
    learning_rate=1e-4,
    fp16=True,
    max_grad_norm=0.3,
    num_train_epochs=4,
    evaluation_strategy="steps",
    eval_steps=0.2,
    warmup_ratio=0.05,
    save_strategy="epoch",
    group_by_length=True,
    output_dir=OUTPUT_DIR,
    report_to="tensorboard",
    save_safetensors=True,
    lr_scheduler_type="cosine",
    seed=42,
)
model.config.use_cache = False  


In [ ]:
from trl import SFTTrainer
trainer = SFTTrainer(
    model=model,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    peft_config=lora_config,
    dataset_text_field="text",
    max_seq_length=2250,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()

In [ ]:
peft_model_path="./fineTunedModel"
trainer.model.save_pretrained(peft_model_path)
tokenizer.save_pretrained(peft_model_path)

In [ ]:

from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

peft_model_dir = "./fineTunedModel"

trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True,
)
tokenizer = AutoTokenizer.from_pretrained(peft_model_dir)

In [ ]:
input_ids = tokenizer(prompt, return_tensors='pt',truncation=True).input_ids.cuda()

In [ ]:

outputs = trained_model.generate(input_ids=input_ids, max_new_tokens=100, )
output= tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]

In [ ]:
print(output)

In [ ]:
trained_model = AutoPeftModelForCausalLM.from_pretrained(
    peft_model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
)

fineTunedModel = trained_model.merge_and_unload()

merged_model.save_pretrained("fineTunedModel",safe_serialization=True)
tokenizer.save_pretrained("fineTunedModel")
